In [2]:
import requests

# Set your API token here
API_TOKEN = "55300f54b2666ce08fd98ace51fee6eb3dea8aa5"

search_query = "The Bengal Districts Act, 1836"
url = f"https://api.indiankanoon.org/search/?formInput={search_query}&pagenum=0&doctypes=highcourts"

headers = {
    "Authorization": f"Token {API_TOKEN}",
    "Accept": "application/json"
}

response = requests.post(url, headers=headers)

print("Status Code:", response.status_code)
print("Response JSON:", response.json())

Status Code: 200
Response JSON: {'categories': [['Filter by AI Tags', [{'value': 'appeal-high-court < writ-petition-high-court', 'formInput': 'The%20Bengal%20Districts%20Act%2C%201836+tag:appeal-high-court < writ-petition-high-court'}, {'value': 'criminal-procedure-code', 'formInput': 'The%20Bengal%20Districts%20Act%2C%201836+tag:criminal-procedure-code'}, {'value': 'protection-of-life-and-liberty', 'formInput': 'The%20Bengal%20Districts%20Act%2C%201836+tag:protection-of-life-and-liberty'}, {'value': 'equality-before-law', 'formInput': 'The%20Bengal%20Districts%20Act%2C%201836+tag:equality-before-law'}, {'value': 'indian-penal-code', 'formInput': 'The%20Bengal%20Districts%20Act%2C%201836+tag:indian-penal-code'}, {'value': 'review-judgment', 'formInput': 'The%20Bengal%20Districts%20Act%2C%201836+tag:review-judgment'}, {'value': 'law-governing-evidence', 'formInput': 'The%20Bengal%20Districts%20Act%2C%201836+tag:law-governing-evidence'}, {'value': 'limitation', 'formInput': 'The%20Bengal

In [7]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

def indian_kannon_search_tool(query: str, pagenum: int = 1):
    """Search Indian Kanoon for case law or statutes or Judgements
    Input: query (str) - search keywords or case name
    Output: top 2 search results (title + link + snippet + truncated judgment)"""

    base_url = "https://api.indiankanoon.org"
    api_key = os.getenv("INDIAN_KANOON_API_KEY")

    headers = {
        "Authorization": f"Token {api_key}",
        "Accept": "application/json"
    }

    # ✅ Use POST instead of GET
    search_url = f"{base_url}/search/"
    payload = {"formInput": query, "pagenum": pagenum}

    url = f"https://api.indiankanoon.org/search/?formInput={search_query}&pagenum=0&doctypes=highcourts"

    response = requests.post(url, headers=headers,)
    if response.status_code != 200:
        return f"Error {response.status_code}: {response.text}"

    data = response.json()
    results = []

    # ✅ Limit to top 2 results
    for item in data.get("docs", [])[:2]:
        docid = item.get("id", "")
        if not docid:
            continue

        # Fetch judgment text (also POST)
        doc_url = f"{base_url}/doc/{docid}/"
        doc_res = requests.post(doc_url, headers=headers)
        if doc_res.status_code != 200:
            judgment_text = f"Could not fetch doc {docid}"
        else:
            judgment_data = doc_res.json()
            judgment_text = judgment_data.get("doc", "No text available")

        results.append({
            "title": item.get("title", "Not specified"),
            "citation": item.get("citation", "Not specified"),
            "link": f"https://indiankanoon.org/doc/{docid}/",
            "snippet": item.get("snippet", "").strip(),
            "judgment_text": judgment_text[:2000] + "..."
        })

    return data if data else "No results found."


if __name__ == "__main__":
    data = (indian_kannon_search_tool("W.Daniel Rajkumar vs V.M.Sivasamy on 5 August, 2022", 1))


In [9]:
data

{'categories': [['Filter by AI Tags',
   [{'value': 'appeal-high-court < writ-petition-high-court',
     'formInput': 'The%20Bengal%20Districts%20Act%2C%201836+tag:appeal-high-court < writ-petition-high-court'},
    {'value': 'criminal-procedure-code',
     'formInput': 'The%20Bengal%20Districts%20Act%2C%201836+tag:criminal-procedure-code'},
    {'value': 'protection-of-life-and-liberty',
     'formInput': 'The%20Bengal%20Districts%20Act%2C%201836+tag:protection-of-life-and-liberty'},
    {'value': 'equality-before-law',
     'formInput': 'The%20Bengal%20Districts%20Act%2C%201836+tag:equality-before-law'},
    {'value': 'indian-penal-code',
     'formInput': 'The%20Bengal%20Districts%20Act%2C%201836+tag:indian-penal-code'},
    {'value': 'review-judgment',
     'formInput': 'The%20Bengal%20Districts%20Act%2C%201836+tag:review-judgment'},
    {'value': 'law-governing-evidence',
     'formInput': 'The%20Bengal%20Districts%20Act%2C%201836+tag:law-governing-evidence'},
    {'value': 'limit

In [19]:
import requests
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup

load_dotenv()

def indian_kannon_search_tool(query: str, pagenum: int = 1):
    """Search Indian Kanoon for case law, statutes or judgments.
    Input: query (str) - search keywords or case name
    Output: top 2 search results with metadata and truncated judgment text
    """

    base_url = "https://api.indiankanoon.org"
    api_key = os.getenv("INDIAN_KANOON_API_KEY")

    headers = {
        "Authorization": f"Token {api_key}",
        "Accept": "application/json"
    }

    # Use POST instead of GET
    search_url = f"{base_url}/search/"
    payload = {"formInput": query, "pagenum": pagenum}

    response = requests.post(search_url, headers=headers, data=payload)
    if response.status_code != 200:
        return f"Error {response.status_code}: {response.text}"

    data = response.json()
    results = []

    # Iterate top 2 docs
    for item in data.get("docs", [])[:2]:
        docid = item.get("tid")  # ✅ correct field
        if not docid:
            continue

        # Fetch full judgment/act
        doc_url = f"{base_url}/doc/{docid}/"
        doc_res = requests.post(doc_url, headers=headers)
        if doc_res.status_code != 200:
            judgment_text = f"Could not fetch doc {docid}"
        else:
            judgment_data = doc_res.json()
            judgment_text = judgment_data.get("doc", "No text available")

        results.append({
            "title": item.get("title", "Not specified"),
            "citation": item.get("citation", "Not specified"),
            "publishdate": item.get("publishdate", "Unknown"),
            "docsource": item.get("docsource", "Unknown"),
            "link": f"https://indiankanoon.org/doc/{docid}/",
            "snippet": item.get("headline", "").strip(),
            "judgment_text": judgment_text + "..."
        })

    return results if results else "No results found."


if __name__ == "__main__":
    res = indian_kannon_search_tool("The Bengal Districts Act, 1836", 1)
    for r in res:
        print(r)


{'title': 'Shib Sankar Majumdar vs Union Of India & Ors on 30 September, 2011', 'citation': 'Not specified', 'publishdate': '2011-09-30', 'docsource': 'Calcutta High Court (Appellete Side)', 'link': 'https://indiankanoon.org/doc/176218423/', 'snippet': 'Certificate issued by Additional Director, Survey, West <b>Bengal</b> ex-officio\n\n        Joint Secretary, Government of West <b>Bengal</b>.\n \n \n\n                                         12 \n                                           13 \n\n\n        (c) Report of the office ... Originally\n\n<b>district</b> 24 Paraganas was one <b>district</b>, thereafter by and under a Notification in\n\nexercise of power conferred by the  <b>Bengal</b> <b>District</b> <b>Act</b>, <b>1836</b>  read', 'judgment_text': '<h2 class="doc_title">Shib Sankar Majumdar vs Union Of India &amp; Ors on 30 September, 2011</h2>\n\n<h3 class="doc_author">Author: <a href="/search/?formInput=authorid:k-j-sengupta">K. J. Sengupta</a></h3>\n\n<h3 class="doc_bench

In [11]:
res

[{'title': 'Shib Sankar Majumdar vs Union Of India & Ors on 30 September, 2011',
  'citation': 'Not specified',
  'publishdate': '2011-09-30',
  'docsource': 'Calcutta High Court (Appellete Side)',
  'link': 'https://indiankanoon.org/doc/176218423/',
  'snippet': 'Certificate issued by Additional Director, Survey, West <b>Bengal</b> ex-officio\n\n        Joint Secretary, Government of West <b>Bengal</b>.\n \n \n\n                                         12 \n                                           13 \n\n\n        (c) Report of the office ... Originally\n\n<b>district</b> 24 Paraganas was one <b>district</b>, thereafter by and under a Notification in\n\nexercise of power conferred by the  <b>Bengal</b> <b>District</b> <b>Act</b>, <b>1836</b>  read',
  'judgment_text': '<h2 class="doc_title">Shib Sankar Majumdar vs Union Of India &amp; Ors on 30 September, 2011</h2>\n\n<h3 class="doc_author">Author: <a href="/search/?formInput=authorid:k-j-sengupta">K. J. Sengupta</a></h3>\n\n<h3 cla

In [20]:
soup = BeautifulSoup(res[0]["judgment_text"],"html.parser")
soup

<h2 class="doc_title">Shib Sankar Majumdar vs Union Of India &amp; Ors on 30 September, 2011</h2>
<h3 class="doc_author">Author: <a href="/search/?formInput=authorid:k-j-sengupta">K. J. Sengupta</a></h3>
<h3 class="doc_bench">Bench: <a href="/search/?formInput=benchid:k-j-sengupta">Kalyan Jyoti Sengupta</a></h3>
<pre id="pre_1">                                           1


                          IN THE HIGH COURT AT CALCUTTA
                              Civil Appellate Jurisdiction
                                     Appellate Side
PRESENT:
The Hon'ble JUSTICE KALYAN JYOTI SENGUPTA
                      AND
The Hon'ble JUSTICE JOYMALYA BAGCHI

                            F.M.A. 2492 of 2010
                                 With
                            F.M.A. 2493 of 2010
                              CAN 5058 of 2011.

                               Shib Sankar Majumdar
                                       Versus
                                Union of India &amp; Ors.
Mr.

In [ ]:
qwe = soup.text.split("\n\n\n")
j=0
for i in qwe:
    print("############     ",j)
    print(len(i))
    print()

    j+=1

############      0
Shib Sankar Majumdar vs Union Of India & Ors on 30 September, 2011
Author: K. J. Sengupta
Bench: Kalyan Jyoti Sengupta
                                           1
############      1
                          IN THE HIGH COURT AT CALCUTTA
                              Civil Appellate Jurisdiction
                                     Appellate Side
PRESENT:
The Hon'ble JUSTICE KALYAN JYOTI SENGUPTA
                      AND
The Hon'ble JUSTICE JOYMALYA BAGCHI

                            F.M.A. 2492 of 2010
                                 With
                            F.M.A. 2493 of 2010
                              CAN 5058 of 2011.

                               Shib Sankar Majumdar
                                       Versus
                                Union of India & Ors.
Mr. Kazi Safiullah
Mr. Debabrata Banerjee...for the appellant/applicant

Mr. P.K. Das
Mr. Subhasis Pal....for respondent No.8.
Ms. Vineeta Meharia
Mr. S. Choudhury
Mr. V. Saha.....

In [23]:
print(soup.text)

Shib Sankar Majumdar vs Union Of India & Ors on 30 September, 2011
Author: K. J. Sengupta
Bench: Kalyan Jyoti Sengupta
                                           1


                          IN THE HIGH COURT AT CALCUTTA
                              Civil Appellate Jurisdiction
                                     Appellate Side
PRESENT:
The Hon'ble JUSTICE KALYAN JYOTI SENGUPTA
                      AND
The Hon'ble JUSTICE JOYMALYA BAGCHI

                            F.M.A. 2492 of 2010
                                 With
                            F.M.A. 2493 of 2010
                              CAN 5058 of 2011.

                               Shib Sankar Majumdar
                                       Versus
                                Union of India & Ors.
Mr. Kazi Safiullah
Mr. Debabrata Banerjee...for the appellant/applicant

Mr. P.K. Das
Mr. Subhasis Pal....for respondent No.8.
Ms. Vineeta Meharia
Mr. S. Choudhury
Mr. V. Saha.......for IOC

Mr. Indrajit Chakraborty...